In [1]:
import mdtraj as md
import numpy as np
import os, math
import pandas as pd 
import matplotlib.pyplot as plt
from utils import binMolDen
from xgboost import XGBRegressor
import torch
from model import *
from solv_utils import *
LJ = {
    'NA': ['0.3526', '2.1600'],
    'CL': ['0.0128', '4.8305'],
    'MG': ['0.8750', '2.1200'],
    'LI': ['0.3367', '1.4094'],
    'K':  ['0.4297', '2.8384']
}

In [2]:
# Load Ground truth
gt = np.load('data/ground_truth.npz', allow_pickle=True)['data'].item()

In [3]:
# Load XGBoost model
bst = XGBRegressor()
bst.load_model('train/XGBoost/xgb_model.json')

In [4]:
# Load Neural network
if torch.cuda.is_available():
    device = 'gpu'
else:
    device = 'cpu'
NN = IonNet(n_in=6, activation='ReLU').to(device)
state_dict = torch.load('train/NN/model.pth', map_location=device)
NN.load_state_dict(state_dict)

<All keys matched successfully>

In [11]:
# binSize=0.05
# ion = 'CL'
# g = 2
# conc = 3

def plot_single(ion, g, conc, binSize, ext=''):
    sig, eps = float(LJ[ion][1]), float(LJ[ion][0])
    if ion == 'CL':
        cha = -1
    elif ion == 'MG':
        cha = 2
    else:
        cha = 1
    # Get ground truth
    xdim, ydim, nIon = gt['%s_%s_%s'%(ion, f2str(g), f2str(conc))][binSize][0]
    cdf_gt = gt['%s_%s_%s'%(ion, f2str(g), f2str(conc))][binSize][1]
    cdf_gt = np.array(cdf_gt)
    pdf_gt = cdf_gt[1:] - cdf_gt[:-1]

    # Get prediction from XGBoost
    bins = np.array([[i*binSize, g, conc, sig, eps, cha] for i in range(int((g/2+0.1)//binSize)+2)])
    cdf_xgb = bst.predict(bins)
    cdf_xgb = np.array(cdf_xgb)
    pdf_xgb= cdf_xgb[1:]-cdf_xgb[:-1]

    # Get prediction from NN
    with torch.no_grad():
        NN.eval()
        cdf_nn = []
        for b in range(int((g/2+0.1)//binSize)+2):
            b=b*binSize
            p = NN(torch.tensor([b, g, conc, sig, eps, cha]).float()).numpy()
            cdf_nn.append(p)
        cdf_nn = np.array(cdf_nn)
        pdf_nn = cdf_nn[1:]-cdf_nn[:-1]
        # conc_profile = binMolDen(3.1928623, 3.4032, binSize, pdf*len(ions)/2)
    mol_gt = binMolDen(xdim, ydim, binSize, pdf_gt*(nIon/2))
    mol_xgb = binMolDen(xdim, ydim, binSize, pdf_xgb*(nIon/2))
    mol_nn = binMolDen(xdim, ydim, binSize, pdf_nn*(nIon/2))
    mol_gt = mol_gt[:len(mol_nn)]
    plt.figure(figsize=[3.5,3], dpi=600)

    plt.plot(bins[:len(mol_gt), 0], mol_gt,marker='o', linewidth=0.5, markersize=1.5, label='MD')
    plt.plot(bins[:-1, 0], mol_xgb, marker='s', linewidth=0.5, markersize=1.5, label='XGBoost')
    plt.plot(bins[:-1, 0], mol_nn, marker='x', linewidth=0.5, markersize=2.5, markeredgewidth=0.6,label='NN')
    plt.xlabel('Distance from channel center (nm)', fontsize=11)
    plt.ylabel('Ion concentration (M)', fontsize= 11)
    plt.legend(frameon=False)
    plt.tick_params(direction='in')
    plt.savefig('figure/preliminary/new/%s_%s_%s%s.png'%(ion, f2str(g), f2str(conc), ext), bbox_inches='tight')
    plt.close()
    return

In [6]:
binSize=0.05
for ion in ['NA', 'CL']:
    for g in [1, 2, 3]:
        for conc in [1, 2, 3]:
            plot_single(ion, g, conc, binSize)

In [7]:
binSize=0.05
for ion in ['NA', 'CL', 'MG', 'K', 'LI']:
    plot_single(ion, 2, 2.2, binSize)

In [14]:
for b in [0.01, 0.02, 0.05, 0.1]:
    plot_single('NA', 2, 2.2, b, '_'+str(b))

In [9]:
plot_single('CL', 1.6, 2.2, 0.05)
plot_single('NA', 1.6, 2.2, 0.05)

In [10]:
0.8//0.05

16.0